# Dataloading 01

In this notebook, we'll figure out how to use PyTorch's DataLoader class to load our massive files without reading the entirety of them into memory

In [1]:
import comet_ml
import dask.dataframe as dd
import pandas as pd 
import torch
import linecache 
import csv
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torch.nn.functional as F
import sys, os
from pathlib import Path
import plotly.express as px 
from sklearn.utils.class_weight import compute_class_weight
import torch

sys.path.append('../src')
sys.path.append('../src/models/lib')
here = Path().cwd()

We'll first design a custom dataset to use with PyTorch's `DataLoader` class

In [2]:
from models.lib.neural import *

In [3]:
primary = GeneExpressionData(
    filename='../data/processed/primary.csv',
    labelname='../data/processed/meta_primary_labels.csv',
    class_label='Subtype'
)

In [4]:
class Model(nn.Module):
    def __init__(self, inputs, outputs):
        super(Model, self).__init__()

        layers = [
            nn.Linear(1024, 1024),
            nn.ReLU(),
            # nn.Dropout(0.5),
            nn.BatchNorm1d(1024, 1024),
        ]

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputs, 1024),
            *layers,
            nn.Linear(1024, outputs),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = Model(
    inputs = primary.num_features(),
    outputs = primary.num_labels(),
)

In [6]:
train_size = int(0.80 * len(primary))
test_size = len(primary) - train_size

train, test = torch.utils.data.random_split(primary, [train_size, test_size])

trainloader = DataLoader(
    train, 
    batch_size=2,
)

validloader = DataLoader(
    test, 
    batch_size=2, 
)

In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


In [9]:
epochs = 5
min_valid_loss = np.inf

for e in range(epochs):
    train_loss = 0.0
    model.train()     # Optional when not using Model Specific layer
    for data, labels in trainloader:
        print(loss)
        optimizer.zero_grad()
        target = model(data)
        loss = criterion(target,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    valid_loss = 0.0
    model.eval() # Optional when not using Model Specific layer
    for data, labels in validloader:
        target = model(data)
        loss = criterion(target,labels)
        valid_loss = loss.item() * data.size(0)

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(trainloader)} \t\t Validation Loss: {valid_loss / len(validloader)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')

tensor(1.7818, grad_fn=<NllLossBackward0>)
tensor(3.2780, grad_fn=<NllLossBackward0>)
tensor(2.8724, grad_fn=<NllLossBackward0>)
tensor(2.6151, grad_fn=<NllLossBackward0>)
tensor(3.5795, grad_fn=<NllLossBackward0>)
tensor(3.8546, grad_fn=<NllLossBackward0>)
tensor(2.8463, grad_fn=<NllLossBackward0>)
tensor(2.2711, grad_fn=<NllLossBackward0>)
tensor(3.1268, grad_fn=<NllLossBackward0>)
tensor(4.5536, grad_fn=<NllLossBackward0>)
tensor(1.7892, grad_fn=<NllLossBackward0>)
tensor(2.7816, grad_fn=<NllLossBackward0>)
tensor(2.9241, grad_fn=<NllLossBackward0>)
tensor(2.2694, grad_fn=<NllLossBackward0>)
tensor(3.4514, grad_fn=<NllLossBackward0>)
tensor(3.4947, grad_fn=<NllLossBackward0>)
tensor(3.8730, grad_fn=<NllLossBackward0>)
tensor(3.3989, grad_fn=<NllLossBackward0>)
tensor(3.6261, grad_fn=<NllLossBackward0>)
tensor(2.6731, grad_fn=<NllLossBackward0>)
tensor(2.3506, grad_fn=<NllLossBackward0>)
tensor(3.0805, grad_fn=<NllLossBackward0>)
tensor(3.3627, grad_fn=<NllLossBackward0>)
tensor(2.39

tensor(4.1568, grad_fn=<NllLossBackward0>)
tensor(2.9650, grad_fn=<NllLossBackward0>)
tensor(4.0574, grad_fn=<NllLossBackward0>)
tensor(2.3564, grad_fn=<NllLossBackward0>)
tensor(2.7719, grad_fn=<NllLossBackward0>)
tensor(2.4207, grad_fn=<NllLossBackward0>)
tensor(1.7640, grad_fn=<NllLossBackward0>)
tensor(2.6288, grad_fn=<NllLossBackward0>)
tensor(3.1793, grad_fn=<NllLossBackward0>)
tensor(2.9968, grad_fn=<NllLossBackward0>)
tensor(1.7625, grad_fn=<NllLossBackward0>)
tensor(2.4609, grad_fn=<NllLossBackward0>)
tensor(3.4735, grad_fn=<NllLossBackward0>)
tensor(3.3488, grad_fn=<NllLossBackward0>)
tensor(2.5613, grad_fn=<NllLossBackward0>)
tensor(3.1302, grad_fn=<NllLossBackward0>)
tensor(3.2291, grad_fn=<NllLossBackward0>)
tensor(3.2759, grad_fn=<NllLossBackward0>)
tensor(2.4619, grad_fn=<NllLossBackward0>)
tensor(2.3560, grad_fn=<NllLossBackward0>)
tensor(2.2400, grad_fn=<NllLossBackward0>)
tensor(2.8507, grad_fn=<NllLossBackward0>)
tensor(3.0562, grad_fn=<NllLossBackward0>)
tensor(3.20

tensor(2.4076, grad_fn=<NllLossBackward0>)
tensor(2.5231, grad_fn=<NllLossBackward0>)
tensor(3.4504, grad_fn=<NllLossBackward0>)
tensor(2.3478, grad_fn=<NllLossBackward0>)
tensor(2.6931, grad_fn=<NllLossBackward0>)
tensor(3.5171, grad_fn=<NllLossBackward0>)
tensor(2.0212, grad_fn=<NllLossBackward0>)
tensor(4.3053, grad_fn=<NllLossBackward0>)
tensor(2.9315, grad_fn=<NllLossBackward0>)
tensor(2.2394, grad_fn=<NllLossBackward0>)
tensor(2.8463, grad_fn=<NllLossBackward0>)
tensor(3.4043, grad_fn=<NllLossBackward0>)
tensor(3.3470, grad_fn=<NllLossBackward0>)
tensor(2.2409, grad_fn=<NllLossBackward0>)
tensor(3.7083, grad_fn=<NllLossBackward0>)
tensor(2.9386, grad_fn=<NllLossBackward0>)
tensor(2.3469, grad_fn=<NllLossBackward0>)
tensor(2.2976, grad_fn=<NllLossBackward0>)
tensor(3.6301, grad_fn=<NllLossBackward0>)
tensor(3.9385, grad_fn=<NllLossBackward0>)
tensor(3.7339, grad_fn=<NllLossBackward0>)
tensor(2.2920, grad_fn=<NllLossBackward0>)
tensor(2.7764, grad_fn=<NllLossBackward0>)
tensor(2.83

tensor(2.6081, grad_fn=<NllLossBackward0>)
tensor(3.1047, grad_fn=<NllLossBackward0>)
tensor(2.9596, grad_fn=<NllLossBackward0>)
tensor(2.9273, grad_fn=<NllLossBackward0>)
tensor(4.4333, grad_fn=<NllLossBackward0>)
tensor(3.3991, grad_fn=<NllLossBackward0>)
tensor(3.2989, grad_fn=<NllLossBackward0>)
tensor(2.9322, grad_fn=<NllLossBackward0>)
tensor(3.2424, grad_fn=<NllLossBackward0>)
tensor(3.3871, grad_fn=<NllLossBackward0>)
tensor(2.7177, grad_fn=<NllLossBackward0>)
tensor(1.7274, grad_fn=<NllLossBackward0>)
tensor(3.0285, grad_fn=<NllLossBackward0>)
tensor(1.7242, grad_fn=<NllLossBackward0>)
tensor(3.5246, grad_fn=<NllLossBackward0>)
tensor(1.7212, grad_fn=<NllLossBackward0>)
tensor(3.3237, grad_fn=<NllLossBackward0>)
tensor(3.2294, grad_fn=<NllLossBackward0>)
tensor(3.0116, grad_fn=<NllLossBackward0>)
tensor(4.2135, grad_fn=<NllLossBackward0>)
tensor(3.5521, grad_fn=<NllLossBackward0>)
tensor(2.3476, grad_fn=<NllLossBackward0>)
tensor(3.0555, grad_fn=<NllLossBackward0>)
tensor(2.07

tensor(3.3489, grad_fn=<NllLossBackward0>)
tensor(2.0811, grad_fn=<NllLossBackward0>)
tensor(2.4481, grad_fn=<NllLossBackward0>)
tensor(3.5642, grad_fn=<NllLossBackward0>)
tensor(3.8397, grad_fn=<NllLossBackward0>)
tensor(3.2600, grad_fn=<NllLossBackward0>)
tensor(2.8991, grad_fn=<NllLossBackward0>)
tensor(3.2657, grad_fn=<NllLossBackward0>)
tensor(3.0920, grad_fn=<NllLossBackward0>)
tensor(2.0208, grad_fn=<NllLossBackward0>)
tensor(3.1198, grad_fn=<NllLossBackward0>)
tensor(2.3380, grad_fn=<NllLossBackward0>)
tensor(4.0167, grad_fn=<NllLossBackward0>)
tensor(2.4462, grad_fn=<NllLossBackward0>)
tensor(4.5967, grad_fn=<NllLossBackward0>)
tensor(3.4705, grad_fn=<NllLossBackward0>)
tensor(3.5595, grad_fn=<NllLossBackward0>)
tensor(3.0387, grad_fn=<NllLossBackward0>)
tensor(3.2631, grad_fn=<NllLossBackward0>)
tensor(3.1216, grad_fn=<NllLossBackward0>)
tensor(2.3834, grad_fn=<NllLossBackward0>)
tensor(2.4234, grad_fn=<NllLossBackward0>)
tensor(2.3916, grad_fn=<NllLossBackward0>)
tensor(2.39

...

tensor(2.5733, grad_fn=<NllLossBackward0>)
tensor(2.4516, grad_fn=<NllLossBackward0>)
tensor(2.2420, grad_fn=<NllLossBackward0>)
tensor(1.8682, grad_fn=<NllLossBackward0>)
tensor(3.1515, grad_fn=<NllLossBackward0>)
tensor(2.9241, grad_fn=<NllLossBackward0>)
tensor(2.5163, grad_fn=<NllLossBackward0>)
tensor(2.7615, grad_fn=<NllLossBackward0>)
tensor(3.3725, grad_fn=<NllLossBackward0>)
tensor(2.5923, grad_fn=<NllLossBackward0>)
tensor(4.8221, grad_fn=<NllLossBackward0>)
tensor(1.9567, grad_fn=<NllLossBackward0>)
tensor(2.8830, grad_fn=<NllLossBackward0>)
tensor(2.1884, grad_fn=<NllLossBackward0>)
tensor(3.4555, grad_fn=<NllLossBackward0>)
tensor(3.3412, grad_fn=<NllLossBackward0>)
tensor(2.3621, grad_fn=<NllLossBackward0>)
tensor(2.0039, grad_fn=<NllLossBackward0>)
tensor(2.5097, grad_fn=<NllLossBackward0>)
tensor(2.9043, grad_fn=<NllLossBackward0>)
tensor(2.7242, grad_fn=<NllLossBackward0>)
tensor(2.4477, grad_fn=<NllLossBackward0>)
tensor(3.5443, grad_fn=<NllLossBackward0>)
tensor(2.61

tensor(2.3845, grad_fn=<NllLossBackward0>)
tensor(1.9293, grad_fn=<NllLossBackward0>)
tensor(2.7181, grad_fn=<NllLossBackward0>)
tensor(2.3361, grad_fn=<NllLossBackward0>)
tensor(1.9014, grad_fn=<NllLossBackward0>)
tensor(4.9225, grad_fn=<NllLossBackward0>)
tensor(2.9992, grad_fn=<NllLossBackward0>)
tensor(3.3084, grad_fn=<NllLossBackward0>)
tensor(2.5041, grad_fn=<NllLossBackward0>)
tensor(3.4549, grad_fn=<NllLossBackward0>)
tensor(2.3639, grad_fn=<NllLossBackward0>)
tensor(2.1599, grad_fn=<NllLossBackward0>)
tensor(2.2074, grad_fn=<NllLossBackward0>)
tensor(1.7994, grad_fn=<NllLossBackward0>)
tensor(1.9769, grad_fn=<NllLossBackward0>)
tensor(3.2021, grad_fn=<NllLossBackward0>)
tensor(2.8907, grad_fn=<NllLossBackward0>)
tensor(2.5074, grad_fn=<NllLossBackward0>)
tensor(2.7898, grad_fn=<NllLossBackward0>)
tensor(4.0061, grad_fn=<NllLossBackward0>)
tensor(1.8251, grad_fn=<NllLossBackward0>)
tensor(2.4145, grad_fn=<NllLossBackward0>)
tensor(3.1549, grad_fn=<NllLossBackward0>)
tensor(3.60

tensor(2.1537, grad_fn=<NllLossBackward0>)
tensor(2.9116, grad_fn=<NllLossBackward0>)
tensor(2.0793, grad_fn=<NllLossBackward0>)
tensor(2.9081, grad_fn=<NllLossBackward0>)
tensor(2.5411, grad_fn=<NllLossBackward0>)
tensor(2.9451, grad_fn=<NllLossBackward0>)
tensor(2.4241, grad_fn=<NllLossBackward0>)
tensor(2.4949, grad_fn=<NllLossBackward0>)
tensor(2.0905, grad_fn=<NllLossBackward0>)
tensor(2.0929, grad_fn=<NllLossBackward0>)
tensor(2.6848, grad_fn=<NllLossBackward0>)
tensor(1.9434, grad_fn=<NllLossBackward0>)
tensor(2.4586, grad_fn=<NllLossBackward0>)
tensor(3.1167, grad_fn=<NllLossBackward0>)
tensor(2.5944, grad_fn=<NllLossBackward0>)
tensor(2.8000, grad_fn=<NllLossBackward0>)
tensor(2.9277, grad_fn=<NllLossBackward0>)
tensor(3.3255, grad_fn=<NllLossBackward0>)
tensor(2.3837, grad_fn=<NllLossBackward0>)
tensor(3.4544, grad_fn=<NllLossBackward0>)
tensor(3.4731, grad_fn=<NllLossBackward0>)
tensor(3.2926, grad_fn=<NllLossBackward0>)
tensor(1.9469, grad_fn=<NllLossBackward0>)
tensor(3.79

tensor(2.2174, grad_fn=<NllLossBackward0>)
tensor(2.1181, grad_fn=<NllLossBackward0>)
tensor(2.9331, grad_fn=<NllLossBackward0>)
tensor(2.5859, grad_fn=<NllLossBackward0>)
tensor(2.0192, grad_fn=<NllLossBackward0>)
tensor(2.5541, grad_fn=<NllLossBackward0>)
tensor(3.3129, grad_fn=<NllLossBackward0>)
tensor(2.1924, grad_fn=<NllLossBackward0>)
tensor(2.1877, grad_fn=<NllLossBackward0>)
tensor(2.3553, grad_fn=<NllLossBackward0>)
tensor(2.7943, grad_fn=<NllLossBackward0>)
tensor(2.8106, grad_fn=<NllLossBackward0>)
tensor(1.8453, grad_fn=<NllLossBackward0>)
tensor(2.1204, grad_fn=<NllLossBackward0>)
tensor(2.5117, grad_fn=<NllLossBackward0>)
tensor(3.6350, grad_fn=<NllLossBackward0>)
tensor(2.9297, grad_fn=<NllLossBackward0>)
tensor(5.5473, grad_fn=<NllLossBackward0>)
tensor(2.7306, grad_fn=<NllLossBackward0>)
tensor(2.6758, grad_fn=<NllLossBackward0>)
tensor(2.8162, grad_fn=<NllLossBackward0>)
tensor(2.5543, grad_fn=<NllLossBackward0>)
tensor(2.1967, grad_fn=<NllLossBackward0>)
tensor(2.86

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 1024])